In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision

In [2]:
VGG16CONV = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']
VGG16FC = [4096,4096]

class VGG_Net(nn.Module):
    def __init__(self, conv_config, fc_config, in_channels=3, num_classes=1000):
        super(VGG_Net, self).__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.conv_layers = self.create_conv_layers(conv_config)
        self.fc_in = 7*7*512
        self.fc_layers = self.create_fc_layers(fc_config)
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0],-1)
        x = self.fc_layers(x)
        
        return x
    
    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels
        
        for c in architecture:
            if type(c)==int:
                out_channels = c
                
                layers.append(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(3,3), stride=(1,1), padding=(1,1)))
                layers.append(nn.BatchNorm2d(c))
                layers.append(nn.ReLU())
                
                in_channels = c
                
            elif c=='M':
                layers.append(nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)))
                
            
        
        return nn.Sequential(*layers)
    
    def create_fc_layers(self, configuration):
        layers = []
        
        input_size = self.fc_in
        for n in configuration:
            output_size = n
            layers.append(nn.Linear(input_size,output_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(p=0.5))
            
            input_size = n
            
        layers.append(nn.Linear(input_size, self.num_classes))
        layers.append(nn.ReLU())

        return nn.Sequential(*layers)
        
        
            
            
        
         

In [3]:
model = VGG_Net(VGG16CONV, VGG16FC, in_channels=3, num_classes=1000)